# Solving Sudoku Puzzles
## Assignment Preamble
Please ensure you carefully read all of the details and instructions on the assignment page, this section, and the rest of the notebook. If anything is unclear at any time please post on the forum or ask a tutor well in advance of the assignment deadline.

In addition to all of the instructions in the body of the assignment below, you must also follow the following technical instructions for all assignments in this unit. *Failure to do so may result in a grade of zero.*
* [At the bottom of the page](#Submission-Test) is some code which checks you meet the submission requirements. You **must** ensure that this runs correctly before submission.
* Do not modify or delete any of the cells that are marked as test cells, even if they appear to be empty.
* Do not duplicate any cells in the notebook – this can break the marking script. Instead, insert a new cell (e.g. from the menu) and copy across any contents as necessary.

Remember to save and backup your work regularly, and double-check you are submitting the correct version.

This notebook is the primary reference for your submission. You may write code in separate `.py` files but it must be clearly imported into the notebook so that it runs without needing to reference those files, and you must explain clearly what functionality is contained in those files (through comments, markdown cells, etc).

As always, **the work you submit for this assignment must be entirely your own.** Do not copy or work with other students. Do not copy answers that you find online. These assignments are designed to help improve your understanding first and foremost – the process of doing the assignment is part of *learning*. They are also used to assess your ability, and so you must uphold academic integrity. Submitting plagiarised work risks your entire place on your degree.

**The pass mark for this assignment is 40%.** We expect that students, on average, will be able to produce a submission which gets a mark between 50-70% within the normal workload allocation for the unit, but this will vary depending on individual backgrounds. Please ask for help if you are struggling.

## Getting Started
For this assignment, you will be writing an agent that can solve sudoku puzzles. You should be familiar with sudoku puzzles from the unit material. You are given a 9x9 grid with some fixed values. To solve the puzzle, the objective is to fill the empty cells of the grid such that the numbers 1 to 9 appear exactly once in each row, column, and 3x3 block of the grid. 

Below is a sample puzzle along with its solution. 

<img src="images/sudoku.png" style="width: 50%;"/>

For this assignment you will need to submit:
1. The implementation for an agent which can solve sudoku puzzles – this notebook
 * You can use any algorithm you like, from the unit material or otherwise
 * Your code will be subject to automated testing, from which grades will be assigned based on whether it can solve sudokus of varying difficulty
 * To get a high grade on this assignment, the speed of your code will also be a factor – the quicker the better
 * There are some sample tests included below, make sure your code is compatible with the format of these tests
2. A text file that explains your approach and the decisions you made in your own words – a readme file
 * Submissions that do not include the written section will receive zero marks – **this part is mandatory**
 * You may write your file in plain text (.txt) or [Markdown](https://www.markdownguide.org/basic-syntax/) (.md)
 * To get top marks on this assignment, as well as getting a high grade from your implementation, you must also demonstrate excellent academic presentation in your written section

### Choice of Algorithm
The choice of algorithm to solve sudoku puzzles is up to you. We expect you will use search techniques from the unit, but you could make something up yourself, or do some independent research to find something else. You will need to evaluate and balance the trade-off between how well suited you think the algorithm is and how difficult it is to write, but there is some advice below.

I suggest you implement *constraint satisfaction* as it is described in the unit material. You can use the code you have previously been given as a guide. A good implementation of a backtracking depth-first search with constraint propagation should be sufficient to get a good grade in the automated tests (roughly 60-70%).

You could also write a successful agent that uses the other search techniques you have seen in the unit so far: basic search, heuristic search, or local search. You may find these easier to implement, though they may perform less well. 

To get a high grade on this assignment will require a particularly efficient implementation of constraint satisfaction, or something which goes beyond the material we have presented. *This is left unguided and is not factored into the unit workload estimates.*

If you choose to implement more than one algorithm, please feel free to include your code and write about it in part two (readme file), but only the code in this notebook will be used in the automated testing.

## Sample Sudoku Puzzles
To get started, the cell below will load in some sample sudoku puzzles for you so you can see the format. There are sudokus provided of multiple difficulties (easier sudokus typically start with more digits provided). The cell below only loads the easiest, but there is another test cell lower in the notebook which will run your code against all of the provided puzzles.

Each sudoku is a 9x9 NumPy array of integers, where zero represents an empty square. Each difficulty comes with 15 sudokus, so when you load the file, it is stored in a 15x9x9 array.

In [1]:
import numpy as np

# Load sudokus
sudoku = np.load("data/very_easy_puzzle.npy")
print("very_easy_puzzle.npy has been loaded into the variable sudoku")
print(f"sudoku.shape: {sudoku.shape}, sudoku[0].shape: {sudoku[0].shape}, sudoku.dtype: {sudoku.dtype}")

# Load solutions for demonstration
solutions = np.load("data/very_easy_solution.npy")
print()

# Print the first 9x9 sudoku...
print("First sudoku:")
print(sudoku[0], "\n")

# ...and its solution
print("Solution of first sudoku:")
print(solutions[0])

very_easy_puzzle.npy has been loaded into the variable sudoku
sudoku.shape: (15, 9, 9), sudoku[0].shape: (9, 9), sudoku.dtype: int8

First sudoku:
[[1 0 4 3 8 2 9 5 6]
 [2 0 5 4 6 7 1 3 8]
 [3 8 6 9 5 1 4 0 2]
 [4 6 1 5 2 3 8 9 7]
 [7 3 8 1 4 9 6 2 5]
 [9 5 2 8 7 6 3 1 4]
 [5 2 9 6 3 4 7 8 1]
 [6 0 7 2 9 8 5 4 3]
 [8 4 3 0 1 5 2 6 9]] 

Solution of first sudoku:
[[1 7 4 3 8 2 9 5 6]
 [2 9 5 4 6 7 1 3 8]
 [3 8 6 9 5 1 4 7 2]
 [4 6 1 5 2 3 8 9 7]
 [7 3 8 1 4 9 6 2 5]
 [9 5 2 8 7 6 3 1 4]
 [5 2 9 6 3 4 7 8 1]
 [6 1 7 2 9 8 5 4 3]
 [8 4 3 7 1 5 2 6 9]]


## Part One
You should write all of your code for solving sudokus below this cell.

You must include a function called `sudoku_solver(sudoku)` which takes one sudoku puzzle (a 9x9 NumPy array) as input, and returns the solved sudoku as another 9x9 NumPy array. This is the function which will be tested. 

In [2]:
import numpy as np
import copy


def cross_product(rows, cols):
    """
    Helper function used to help create constants used for convenience lookups by the algorithm
    :param rows: row identifiers for the sudoku problem
    :param cols: column identifiers for the sudoku problem
    :return: returns a list of square mapping such as A1, B1 etc...
    """
    return [r+c for r in rows for c in cols]


"""
Constant values defined here for use by application
"""
rows_key = 'ABCDEFGHI'
domain_values = cols_key = '123456789'

# assign alpha numeric keys to each square in the 9x9 grid
squares = cross_product(rows_key, cols_key)

# create a list of column keys
columns_list = [cross_product(rows_key, c) for c in cols_key]
# create a list of row keys
rows_list = [cross_product(r, cols_key) for r in rows_key]
# create a list of 3x3 sectors
sector_list = [cross_product(
    row_sector_identifiers, column_sector_identifiers
) for row_sector_identifiers in ('ABC', 'DEF', 'GHI') for column_sector_identifiers in ('123', '456', '789')]
# create a single collection of rows, columns and sectors
mappings = (columns_list + rows_list + sector_list)

# create a dictionary where the key is each square and the value is a list of rows, columns and sectors which
# relate to the key
row_column_sector = dict((square, [mapping for mapping in mappings if square in mapping]) for square in squares)
# create a dictionary for the constraint satisfaction propagation combining row, column and sector values
# using sets to remove duplicates and making sure to remove itself from the values
propagation_dictionary = dict(
    (square, set(sum(row_column_sector[square], [])) - {square}) for square in squares)


"""
Static functions used by the algorithm
"""


def forward_check(key, value, remaining_values):
    """
    Check remaining values and prune the tree of irrelevant branches
    :param key: current choice key
    :param value: current choice value
    :param remaining_values: search space
    :return: remaining_vluaes with a reduced search space
    """
    rv_pruning = copy.copy(remaining_values)
    for relative_key in propagation_dictionary[key]:
        if relative_key in rv_pruning and value in rv_pruning[relative_key]:
            if len(rv_pruning[relative_key]) == 1:
                return None
            if len(rv_pruning[relative_key]) > 1:
                rv_pruning[relative_key] = rv_pruning[relative_key].replace(value, '')

    return sort_remaining_values(rv_pruning)


def populate_solution(solution, sudoku):
    """
    insert sudoku starting values into solution
    :param solution: solution to populate
    :param sudoku: initial problem
    """
    for row_index, row in enumerate(sudoku):
        for col_index, col in enumerate(row):
            if col > 0:
                square_key = rows_key[row_index] + cols_key[col_index]
                # if the starting solution has a value, apply it to the work in progress solution
                solution[square_key] = str(col)


def constraint_propagation(sudoku):
    """
    Propagate values according to sudoku constraints
    :param sudoku: problem to solve
    :return: solution or None if there is no solution found
    """

    # initialise starting domain values for the solution
    solution = dict((s, domain_values) for s in squares)
    populate_solution(solution, sudoku)

    # then remove the value from the available domain options of all corresponding squares
    for key in squares:
        if len(solution[key]) == 1:
            # only 1 char means the square is set, therefore remove value from associated squares
            for prop_key in propagation_dictionary[key]:
                if solution[key] in solution[prop_key]:
                    # if when removing a character it is the only character available in the domain of the square,
                    # then the initial state must be invalid
                    if len(solution[prop_key]) == 1:
                        # invalid starting state
                        return None
                    solution[prop_key] = solution[prop_key].replace(solution[key], '')
    return solution


def sort_remaining_values(remaining_values):
    """
    Function to sort the remaining values dictionary by the length of the values.
    Each value string contains domain values (1 to 9) with invalid values already removed
    during the constraint satisfaction step. As the key is to fail as soon as possible, we put
    the keys with the least available options to the start, therefore pruning the tree branches early
    during traversal.
    :param remaining_values: list of key, value pairs where the value contains available domain values
    :return: the sorted remaining_values list
    """
    remaining_values = sorted(remaining_values.items(), key=lambda kv: (len(kv[1]), kv[0]))
    sorted_remaining_values = {}
    for rv in remaining_values:
        sorted_remaining_values[rv[0]] = rv[1]
    return sorted_remaining_values


def produce_invalid_solution_grid():
    """
    Helper function to produce an invalid solution grid, which is a 9x9 grid where all values are -1
    :return: Invalid solution grid
    """
    invalid_solution_values = [-1 for k in range(9) for j in range(9)]
    return [invalid_solution_values[m:m + 9] for m in range(0, len(invalid_solution_values), 9)]


"""
End Static functions used by the algorithm
"""


class PartialSudokuState:
    """
    PatrialSudokuState holds the current state of the solution at each node
    while traversing a tree using depth first search
    """

    def __init__(self, solution, remaining_values):
        """
        Initialise the PartialSudokuState to represent a node in the search tree
        :param solution:
        :param remaining_values:
        """
        self.solution = solution
        self.remaining_values = remaining_values
        self.visited = []
        self.continue_flag = True

    def is_complete(self):
        """
        if each square has only 1 value assigned, then we can assume that the problem is solved
        :return: True or False
        """
        return all(len(self.solution[s]) == 1 for s in squares)

    def solve(self):
        """
        Main recursive solve function used for the depth first search approach. Evaluates each node an decides whether 
        to proceed further down the tree or if the current path is a dead end
        :return: Either a complete solution or None in the case of a dead end reached
        """

        # initialise the state variable to None
        state = None

        for key in self.remaining_values:
            for char in self.remaining_values[key]:

                if state is not None and state.is_complete():
                    # already complete, return up the call stack
                    return state

                # check to see if the node has been visited
                node = {key: char}
                if node not in self.visited:
                    self.visited.append(node)
                    partial_solution = self.set_value(key, char)

                    if partial_solution is not None:
                        if partial_solution.is_complete():
                            return partial_solution
                        else:
                            partial_solution.remaining_values = forward_check(key, char, self.remaining_values)
                            # check to see if a conflict was found during the forward check process
                            if partial_solution.remaining_values is None:
                                return None

                            del partial_solution.remaining_values[key]
                            partial_solution.visited = copy.copy(self.visited)
                            state = partial_solution.solve()
                    else:
                        # conflict found, remove from available domain values to ensure more efficient backtracking
                        self.remaining_values[key] = self.remaining_values[key].replace(char, '')

            if state is None:
                # enforce backtracking by returning None
                return None

    def set_value(self, key, value):
        """
        Atempts to set a value in the potential solution. If successful, the function returns a new node of the tree
        :param key: The key identifier for a single square on the sudoku board
        :param value: the value to try and set in the square
        :return: Either a PartialSudokuState or None if there is a conflict or no value available in the domain
        """
        if value in self.solution[key]:
            # create a copy of the solution so as to not affect the current node
            solution = copy.copy(self.solution)
            solution[key] = value
            for prop_key in propagation_dictionary[key]:
                if value in solution[prop_key]:
                    if len(solution[prop_key]) == 1:
                        # only 1 value here so we must have a conflict
                        return None
                    else:
                        solution[prop_key] = solution[prop_key].replace(value, '')
            return PartialSudokuState(solution, self.remaining_values)
        else:
            # value not available within domain
            return None


def sudoku_solver(sudoku):
    """
    Solves a Sudoku puzzle and returns its unique solution.

    Input
        sudoku : 9x9 numpy array
            Empty cells are designated by 0.

    Output
        9x9 numpy array of integers
            It contains the solution, if there is one. If there is no solution, all array entries should be -1.
    """

    solution = constraint_propagation(sudoku)

    # check to see if an invalid starting puzzle has been discovered
    if solution is None:
        return produce_invalid_solution_grid()

    # now we check to see if the solution is complete
    if not all(len(solution[s]) == 1 for s in squares):
        # solution is not solved just by constraint propagation, therefore
        # we must progress to depth first search method

        # find the remaining unanswered squares for the solution
        remaining_values = {}
        for key in solution:
            if len(solution[key]) > 1:
                remaining_values[key] = solution[key]

        if remaining_values:
            # before we progress to depth first search, sort the remaining domain values
            sorted_remaining_values = sort_remaining_values(remaining_values)

            partial_solution = PartialSudokuState(solution, sorted_remaining_values)

            result = partial_solution.solve()
            if result is None:
                return produce_invalid_solution_grid()
            solution = result.solution

    # convert dictionary into numpy array
    solution_values_list = [int(x) for x in list(solution.values())]

    # split array in rows
    solved_sudoku = [solution_values_list[i:i + 9] for i in range(0, len(solution_values_list), 9)]

    return solved_sudoku

All of your code must go above this cell. You may add additional cells into the notebook if you wish, but do not duplicate or copy/paste cells as this can interfere with the grading script.

### Testing Details
There are four difficulties of sudoku provided: very easy, easy, medium, and hard. There are 15 sample sudokus in each category, with solutions as well. Difficulty was determined using reference solvers, but your code may vary; it is conceivable that your code will find some sudokus much easier or harder within a given category, or even between categories.

*All categories that are easy and above will contain* ***invalid initial states***, that is, sudoku puzzles with no solution. In this case, your function should return a 9x9 NumPy array whose values are all equal to -1.

When we test your code, we will firstly test it on the *same* very easy puzzles that you have been given. Then we will test it on additional *hidden* sudokus from each difficulty in turn, easy and up. Grades are awarded based on whether your code can solve the puzzles. For high grades on the hard puzzles, execution time will also be a factor. 

All puzzles must take under 30 seconds each on the test machine to count as successful, but you should be aiming for an average of under a second per puzzle. Hardware varies, but all tests will take place on the same modern desktop machine. Our ‘standard constraint satisfaction’ implementation takes about 0.001 seconds per puzzle for the very easy category, but struggles to solve some of the hard puzzles within the time limit.

***The hard sudokus are labelled as hard for a reason.*** We expect most submissions will not be able to solve them in a reasonable length of time. Use the stop button (■) on the toolbar if you need to terminate your code because it is taking too long.

The best way to improve the performance of your code is through a detailed understanding and smart choice of AI algorithms. This assignment is ***not*** meant to test your ability to write multi-threaded code or any other kind of high-performance code optimisations. 

#### Test Cell
The following code will run your solution over the provided sudoku puzzles. To enable it, set the constant `SKIP_TESTS` to `False`. If you fail any tests of one difficulty, the code will stop, but you can modify this behaviour if you like.

**IMPORTANT**: you must set `SKIP_TESTS` back to `True` before submitting this file!

In [3]:
SKIP_TESTS = False

if not SKIP_TESTS:
    import time
    difficulties = ['very_easy', 'easy', 'medium', 'hard']

    for difficulty in difficulties:
        print(f"Testing {difficulty} sudokus")
        
        sudokus = np.load(f"data/{difficulty}_puzzle.npy")
        solutions = np.load(f"data/{difficulty}_solution.npy")
        
        count = 0
        total_start_time = time.process_time()
        for i in range(len(sudokus)):
            sudoku = sudokus[i].copy()
            print(f"This is {difficulty} sudoku number", i)
            print(sudoku)
            
            start_time = time.process_time()
            your_solution = sudoku_solver(sudoku)
            end_time = time.process_time()
            
            print(f"This is your solution for {difficulty} sudoku number", i)
            print(your_solution)
            
            print("Is your solution correct?")
            if np.array_equal(your_solution, solutions[i]):
                print("Yes! Correct solution.")
                count += 1
            else:
                print("No, the correct solution is:")
                print(solutions[i])
            
            print("This sudoku took", end_time-start_time, "seconds to solve.\n")

        print(f"{count}/{len(sudokus)} {difficulty} sudokus correct")
        if count < len(sudokus):
            break
        total_end_time = time.process_time()
    print("All sudoku took", total_end_time-total_start_time, "seconds to solve.\n")

Testing very_easy sudokus
This is very_easy sudoku number 0
[[1 0 4 3 8 2 9 5 6]
 [2 0 5 4 6 7 1 3 8]
 [3 8 6 9 5 1 4 0 2]
 [4 6 1 5 2 3 8 9 7]
 [7 3 8 1 4 9 6 2 5]
 [9 5 2 8 7 6 3 1 4]
 [5 2 9 6 3 4 7 8 1]
 [6 0 7 2 9 8 5 4 3]
 [8 4 3 0 1 5 2 6 9]]
This is your solution for very_easy sudoku number 0
[[1, 7, 4, 3, 8, 2, 9, 5, 6], [2, 9, 5, 4, 6, 7, 1, 3, 8], [3, 8, 6, 9, 5, 1, 4, 7, 2], [4, 6, 1, 5, 2, 3, 8, 9, 7], [7, 3, 8, 1, 4, 9, 6, 2, 5], [9, 5, 2, 8, 7, 6, 3, 1, 4], [5, 2, 9, 6, 3, 4, 7, 8, 1], [6, 1, 7, 2, 9, 8, 5, 4, 3], [8, 4, 3, 7, 1, 5, 2, 6, 9]]
Is your solution correct?
Yes! Correct solution.
This sudoku took 0.0006269999999999332 seconds to solve.

This is very_easy sudoku number 1
[[0 9 3 1 5 2 6 0 8]
 [8 6 2 7 0 3 1 9 5]
 [1 5 7 9 8 6 3 2 4]
 [9 7 8 4 2 1 0 3 6]
 [5 0 6 8 3 9 4 1 7]
 [3 4 1 5 6 7 2 8 9]
 [6 1 4 2 7 8 9 5 3]
 [7 3 9 6 1 5 8 4 2]
 [2 8 5 3 9 4 7 6 1]]
This is your solution for very_easy sudoku number 1
[[4, 9, 3, 1, 5, 2, 6, 7, 8], [8, 6, 2, 7, 4, 3, 1, 9

This is your solution for hard sudoku number 5
[[8, 1, 2, 7, 6, 5, 9, 3, 4], [4, 5, 3, 2, 1, 9, 6, 8, 7], [9, 7, 6, 4, 8, 3, 1, 5, 2], [1, 2, 9, 6, 3, 8, 4, 7, 5], [5, 3, 4, 1, 2, 7, 8, 9, 6], [7, 6, 8, 5, 9, 4, 2, 1, 3], [3, 8, 5, 9, 4, 6, 7, 2, 1], [6, 9, 1, 3, 7, 2, 5, 4, 8], [2, 4, 7, 8, 5, 1, 3, 6, 9]]
Is your solution correct?
Yes! Correct solution.
This sudoku took 0.08398300000000003 seconds to solve.

This is hard sudoku number 6
[[0 8 0 4 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 2]
 [0 5 0 7 0 8 0 9 0]
 [0 4 0 0 2 0 0 3 5]
 [0 0 0 0 6 0 0 0 0]
 [0 0 8 0 7 0 0 0 9]
 [0 0 2 3 0 0 7 8 0]
 [9 0 0 0 0 5 0 0 0]
 [0 0 0 0 0 0 0 0 0]]
This is your solution for hard sudoku number 6
[[1, 8, 9, 4, 3, 2, 5, 6, 7], [4, 3, 7, 9, 5, 6, 8, 1, 2], [2, 5, 6, 7, 1, 8, 4, 9, 3], [7, 4, 1, 8, 2, 9, 6, 3, 5], [3, 9, 5, 1, 6, 4, 2, 7, 8], [6, 2, 8, 5, 7, 3, 1, 4, 9], [5, 6, 2, 3, 9, 1, 7, 8, 4], [9, 7, 4, 6, 8, 5, 3, 2, 1], [8, 1, 3, 2, 4, 7, 9, 5, 6]]
Is your solution correct?
Yes! Correct solution.
This sudo

## Submission Test
The following cell tests if your notebook is ready for submission. **You must not skip this step!**

Restart the kernel and run the entire notebook (Kernel → Restart & Run All). Now look at the output of the cell below. 

*If there is no output, then your submission is not ready.* Either your code is still running (did you forget to skip tests?) or it caused an error.

As previously mentioned, failing to follow these instructions can result in a grade of zero.

In [6]:
import sys
import pathlib

fail = False;

if not SKIP_TESTS:
    fail = True;
    print("You must set the SKIP_TESTS constant to True in the cell above.")
    
p1 = pathlib.Path('./readme.txt')
p2 = pathlib.Path('./readme.md')
if not (p1.is_file() or p2.is_file()):
    fail = True;
    print("You must include a separate file called readme.txt or readme.md in your submission.")
    
p3 = pathlib.Path('./sudoku.ipynb')
if not p3.is_file():
    fail = True
    print("This notebook file must be named sudoku.ipynb")
    
if "sudoku_solver" not in dir():
    fail = True;
    print("You must include a function called sudoku_solver which accepts a numpy array.")
else: 
    sudoku = np.load("data/very_easy_puzzle.npy")[0]
    solution = np.load("data/very_easy_solution.npy")[0]

    if not np.array_equal(sudoku_solver(sudoku), solution):
        print("Warning:")
        print("Your sudoku_solver function does not correctly solve the first sudoku.")
        print()
        print("Your assignment is unlikely to get any marks from the autograder. While we will")
        print("try to check it manually to assign some partial credit, we encourage you to ask")
        print("for help on the forum or directly to a tutor.")
        print()
        print("Please use the readme file to explain your code anyway.")
    
if fail:
    print()
    sys.stderr.write("Your submission is not ready! Please read and follow the instructions above.")
else:
    print("All checks passed. When you are ready to submit, upload the notebook and readme file to the")
    print("assignment page, without changing any filenames.")
    print()
    print("If you need to submit multiple files, you can archive them in a .zip file. (No other format.)")

All checks passed. When you are ready to submit, upload the notebook and readme file to the
assignment page, without changing any filenames.

If you need to submit multiple files, you can archive them in a .zip file. (No other format.)


In [ ]:
# This is a TEST CELL. Do not delete or change.